In [2]:
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import os
import numpy as np
import torchvision.transforms as transforms
from PIL import Image

In [3]:
class Unet_dataset(Dataset):
    def __init__(self, img_path, label_path, transform=None):
        super().__init__()
        self.dataset = os.listdir(img_path)
        self.labels = os.listdir(label_path)
        self.transform = transform
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        image = Image.open(os.path.join('./unet_train/bldg/src/', self.dataset[idx]))
        label = Image.open(os.path.join('./unet_train/bldg/label/', self.labels[idx]))
        label = torch.from_numpy(np.array(label)).float().unsqueeze(0)
        if self.transform is not None:
            image = self.transform(image)
        
        return image, label

transform = transforms.Compose([
    transforms.ToTensor(),
])
bldg_dataset = Unet_dataset('./unet_train/bldg/src/', './unet_train/bldg/label/', transform=transform)
bldg_dataloader = DataLoader(bldg_dataset, batch_size=8, num_workers=0, shuffle=True)

In [4]:
class BasicConv2d(nn.Module):
    def __init__(self, inp, oup):
        super(BasicConv2d, self).__init__()
        self.conv1 = nn.Conv2d(inp, oup, 3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(oup, oup, 3, stride=1, padding=1)
        self.bn = nn.BatchNorm2d(oup)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn(x)
        x = F.elu(x, inplace=True)
        x = self.conv2(x)
        x = self.bn(x)
        return F.elu(x, inplace=True)

    
class Unet(nn.Module):
    def __init__(self, num_classes=1):
        super(Unet, self).__init__()
        self.conv1 = BasicConv2d(3, 32)
        self.conv2 = BasicConv2d(32, 64)
        self.conv3 = BasicConv2d(64, 128)
        self.conv4 = BasicConv2d(128, 256)
        self.conv5 = BasicConv2d(256, 512)
        
        self.conv6 = BasicConv2d(768, 256)
        self.conv7 = BasicConv2d(384, 128)
        self.conv8 = BasicConv2d(192, 64)
        self.conv9 = BasicConv2d(96, 32)
        
        self.MaxPool = nn.MaxPool2d(2, 2)
        
        self.conv10 = nn.Conv2d(32, num_classes, 1)
    
    def forward(self, x):
        dec1 = self.conv1(x)
        max_dec1 = self.MaxPool(dec1)
        dec2 = self.conv2(max_dec1)
        max_dec2 = self.MaxPool(dec2)
        dec3 = self.conv3(max_dec2)
        max_dec3 = self.MaxPool(dec3)
        dec4 = self.conv4(max_dec3)
        max_dec4 = self.MaxPool(dec4)
        
        center = self.conv5(dec4)
        up6 = torch.cat([
            dec4,  F.upsample_bilinear(center, dec4.size()[2:])], 1)
        dec6 = self.conv6(up6)
        up7 = torch.cat([
            dec3, F.upsample_bilinear(dec6, dec3.size()[2:])], 1)
        dec7 = self.conv7(up7)
        up8 = torch.cat([
            dec2, F.upsample_bilinear(dec7, dec2.size()[2:])], 1)
        dec8 = self.conv8(up8)
        up9 = torch.cat([
            dec1, F.upsample_bilinear(dec8, dec1.size()[2:])], 1)
        dec9 = self.conv9(up9)
        out = self.conv10(dec9)
        return nn.Sigmoid()(out)

In [4]:
# train
from torch.optim import lr_scheduler
from torch.autograd import Variable

model = Unet().cuda()
optimizer = torch.optim.Adam(model.parameters(), 1e-3, (0.9, 0.999), eps=1e-08, weight_decay=1e-4)
criterion = torch.nn.BCELoss()
num_epochs = 20
lambda1 = lambda epoch: pow((1-((epoch-1)/num_epochs)),0.9)
scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda1)


for epoch in range(num_epochs):
    print("---training -epoch", epoch, "---")
    scheduler.step(epoch)
    epoch_loss = []
    acc = []
    usedLr = 0
    
    for param_group in optimizer.param_groups:
        print("LEARNING RATE: ", param_group['lr'])
        usedLr = float(param_group['lr'])
    
    model.train()
    for i, (images, labels) in enumerate(bldg_dataloader):
        l = len(bldg_dataloader)
        images = images.to('cuda').float()
        targets = labels.to('cuda').float()
        
        outputs = model(images)
        loss = criterion(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss.append(loss.item())

        total = len(outputs.view(-1))
        targets = targets.squeeze()
        outputs = outputs.squeeze() > 0.5
        outputs = outputs.float()
        correct = len(outputs[targets==outputs])
        acc_ = correct / total
        
        acc.append(acc_)
        
        if i % 49 == 0:
            average = sum(epoch_loss) / len(epoch_loss)
            accuracy = sum(acc) / len(acc)
            print("loss: {} epoch: {}, acc: {}, {}/{}".format(average, epoch, accuracy, i, l))
    
    # save model
    if epoch % 2 == 0:
        torch.save(model, './model_{}.pth'.format(epoch))

---training -epoch 0 ---
LEARNING RATE:  0.0010448895099824027


D:\anaconda3\envs\pytorch\lib\site-packages\torch\optim\lr_scheduler.py:82: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
D:\anaconda3\envs\pytorch\lib\site-packages\torch\nn\functional.py:2562: UserWarning: nn.functional.upsample_bilinear is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample_bilinear is deprecated. Use nn.functional.interpolate instead.")


loss: 0.7344312071800232 epoch: 0, acc: 0.4294583067602041, 0/6250
loss: 0.6462302422523498 epoch: 0, acc: 0.6548760363520408, 49/6250
loss: 0.6016978811133992 epoch: 0, acc: 0.6964160901102867, 98/6250
loss: 0.5779351022195172 epoch: 0, acc: 0.7127094384523238, 147/6250
loss: 0.5606895848579213 epoch: 0, acc: 0.7225204336572829, 196/6250
loss: 0.5445439551419359 epoch: 0, acc: 0.7316498364780364, 245/6250
loss: 0.5340236428430525 epoch: 0, acc: 0.7374241314640263, 294/6250
loss: 0.5302544780248819 epoch: 0, acc: 0.7391965456013434, 343/6250
loss: 0.522740699104377 epoch: 0, acc: 0.7437787182501816, 392/6250
loss: 0.5199420622031613 epoch: 0, acc: 0.7456528810442445, 441/6250
loss: 0.5174453810858872 epoch: 0, acc: 0.7468168665442247, 490/6250
loss: 0.5135858945272587 epoch: 0, acc: 0.7491275155895691, 539/6250
loss: 0.510937422868149 epoch: 0, acc: 0.7496106743312775, 588/6250
loss: 0.5079553439325674 epoch: 0, acc: 0.7508119225967227, 637/6250
loss: 0.5061397598473439 epoch: 0, acc: 

loss: 0.463563850173669 epoch: 0, acc: 0.7730996644534734, 5782/6250
loss: 0.4633402817426518 epoch: 0, acc: 0.7732097796278362, 5831/6250
loss: 0.46343648011810906 epoch: 0, acc: 0.7731371645604445, 5880/6250
loss: 0.46335949621102057 epoch: 0, acc: 0.7730794584512766, 5929/6250
loss: 0.4633703155872674 epoch: 0, acc: 0.773056222928208, 5978/6250
loss: 0.46308138104577135 epoch: 0, acc: 0.7732321908368939, 6027/6250
loss: 0.4631103902996342 epoch: 0, acc: 0.7731693095889107, 6076/6250
loss: 0.46299445219739294 epoch: 0, acc: 0.7732459558221857, 6125/6250
loss: 0.46305017459971703 epoch: 0, acc: 0.7732143986769813, 6174/6250
loss: 0.4629560588071348 epoch: 0, acc: 0.7732673838346094, 6223/6250


D:\anaconda3\envs\pytorch\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Unet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
D:\anaconda3\envs\pytorch\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type BasicConv2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


...

loss: 0.34730891718435014 epoch: 15, acc: 0.83883881599134, 3283/6250
loss: 0.34738395819113554 epoch: 15, acc: 0.8387589396694769, 3332/6250
loss: 0.34748763312370307 epoch: 15, acc: 0.8387889678597447, 3381/6250
loss: 0.34773203359031357 epoch: 15, acc: 0.8386697972074465, 3430/6250
loss: 0.3475887029570923 epoch: 15, acc: 0.8387202048788261, 3479/6250
loss: 0.34772172998600015 epoch: 15, acc: 0.8386499414382143, 3528/6250
loss: 0.34789641407230437 epoch: 15, acc: 0.8383647943810173, 3577/6250
loss: 0.34821046120146926 epoch: 15, acc: 0.8381830255817689, 3626/6250
loss: 0.3483125711624005 epoch: 15, acc: 0.8382006761661409, 3675/6250
loss: 0.3483561625236633 epoch: 15, acc: 0.8381830720962878, 3724/6250
loss: 0.3479690012469232 epoch: 15, acc: 0.8384179923024949, 3773/6250
loss: 0.3482718855481699 epoch: 15, acc: 0.8382427389463007, 3822/6250
loss: 0.3485186051073184 epoch: 15, acc: 0.8380476291635259, 3871/6250
loss: 0.34864419784413037 epoch: 15, acc: 0.8379245530102815, 3920/6250


loss: 0.3441707319500854 epoch: 16, acc: 0.8398377002712448, 2646/6250
loss: 0.34384281026304475 epoch: 16, acc: 0.8401748064912394, 2695/6250
loss: 0.3438088903227356 epoch: 16, acc: 0.8402710691517039, 2744/6250
loss: 0.3437411941897255 epoch: 16, acc: 0.8401780278872916, 2793/6250
loss: 0.3440616526734313 epoch: 16, acc: 0.8400351900949876, 2842/6250
loss: 0.3437713110796273 epoch: 16, acc: 0.8401314716595927, 2891/6250
loss: 0.3438718987870119 epoch: 16, acc: 0.8399730805436341, 2940/6250
loss: 0.3437201032944547 epoch: 16, acc: 0.8400973056446666, 2989/6250
loss: 0.34307813216595323 epoch: 16, acc: 0.8404519447910588, 3038/6250
loss: 0.34330029468356094 epoch: 16, acc: 0.8404565960203514, 3087/6250
loss: 0.34359753738695703 epoch: 16, acc: 0.8403559699376852, 3136/6250
loss: 0.3440842622796358 epoch: 16, acc: 0.8402169123004678, 3185/6250
loss: 0.34397762333729537 epoch: 16, acc: 0.8402907237928997, 3234/6250
loss: 0.34384747374752916 epoch: 16, acc: 0.8402897986663088, 3283/6250


loss: 0.33637460110273526 epoch: 17, acc: 0.844754543608488, 2009/6250
loss: 0.33619119630038247 epoch: 17, acc: 0.8447954748580768, 2058/6250
loss: 0.33594555937220294 epoch: 17, acc: 0.8449609178348755, 2107/6250
loss: 0.335737031991138 epoch: 17, acc: 0.8449576406210082, 2156/6250
loss: 0.3359942301391799 epoch: 17, acc: 0.8448854004890994, 2205/6250
loss: 0.3357068554855503 epoch: 17, acc: 0.8449949481594186, 2254/6250
loss: 0.33584858463533845 epoch: 17, acc: 0.8448503239084527, 2303/6250
loss: 0.3360534505708847 epoch: 17, acc: 0.8447961263898877, 2352/6250
loss: 0.33667839814805667 epoch: 17, acc: 0.8444794805216518, 2401/6250
loss: 0.33672672352076843 epoch: 17, acc: 0.8445826992535833, 2450/6250
loss: 0.3368370159983635 epoch: 17, acc: 0.844632158801019, 2499/6250
loss: 0.33708358752746215 epoch: 17, acc: 0.8445761386768309, 2548/6250
loss: 0.33686104924450294 epoch: 17, acc: 0.844680090208715, 2597/6250
loss: 0.33708383560090593 epoch: 17, acc: 0.8445042133217315, 2646/6250
l

loss: 0.33491013381421264 epoch: 18, acc: 0.8451181702025762, 1372/6250
loss: 0.33418115702590023 epoch: 18, acc: 0.8454532871297243, 1421/6250
loss: 0.3333491735935292 epoch: 18, acc: 0.8459360847234446, 1470/6250
loss: 0.33421992839461095 epoch: 18, acc: 0.845758633557246, 1519/6250
loss: 0.3347911174231612 epoch: 18, acc: 0.8456815438591779, 1568/6250
loss: 0.3338806488227712 epoch: 18, acc: 0.8460637961051997, 1617/6250
loss: 0.33499050049650936 epoch: 18, acc: 0.8454744219364029, 1666/6250
loss: 0.33468810372089314 epoch: 18, acc: 0.8456647344935564, 1715/6250
loss: 0.334759922815281 epoch: 18, acc: 0.8456825458804265, 1764/6250
loss: 0.33551859865098566 epoch: 18, acc: 0.8454880935974158, 1813/6250
loss: 0.33606912453633386 epoch: 18, acc: 0.844965324847732, 1862/6250
loss: 0.3367146400921826 epoch: 18, acc: 0.8445009497531029, 1911/6250
loss: 0.336422936796139 epoch: 18, acc: 0.8446250534172741, 1960/6250
loss: 0.33702495278633054 epoch: 18, acc: 0.8444762495716565, 2009/6250
lo

loss: 0.3299789134838173 epoch: 19, acc: 0.8482279874840565, 735/6250
loss: 0.3293006064880426 epoch: 19, acc: 0.8486177572143514, 784/6250
loss: 0.32911317126392176 epoch: 19, acc: 0.848688945845875, 833/6250
loss: 0.3285139731699991 epoch: 19, acc: 0.8488846666303138, 882/6250
loss: 0.32943156442321164 epoch: 19, acc: 0.8488619202039728, 931/6250
loss: 0.32943622562103925 epoch: 19, acc: 0.8486034560995661, 980/6250
loss: 0.3306377911958301 epoch: 19, acc: 0.8482663645327679, 1029/6250
loss: 0.33045768574142587 epoch: 19, acc: 0.8483766428547792, 1078/6250
loss: 0.32996792047687457 epoch: 19, acc: 0.8485476424404541, 1127/6250
loss: 0.3309942159038939 epoch: 19, acc: 0.847972697701373, 1176/6250
loss: 0.3308717232154983 epoch: 19, acc: 0.847892864100423, 1225/6250
loss: 0.3306317900091994 epoch: 19, acc: 0.8479420283738505, 1274/6250
loss: 0.33092625377266427 epoch: 19, acc: 0.847861727625105, 1323/6250
loss: 0.3310939280420694 epoch: 19, acc: 0.8478253343569324, 1372/6250
loss: 0.33

In [17]:
from skimage import io
import matplotlib.pyplot as plt

class test_dataset(Dataset):
    def __init__(self, img_path, transform=None):
        super().__init__()
        self.dataset = os.listdir(img_path)
        self.transform = transform
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        image = Image.open(os.path.join('./unet_train/test/', self.dataset[idx]))
        if self.transform is not None:
            image = self.transform(image)
        
        return image


palette = {0: (0, 0, 0),        # Undefined (black)
           1: (0, 255, 0),      # Trees (green)
           2: (0, 0, 255),      # Buildings (red)
           3: (255, 0, 0),      # Water (blue)
           4: (0, 255, 255)}    # Roads (yellow)
    
    
def convert_to_color(arr_2d, palette=palette):
    # Numeric labels to RGB-color encoding
    arr_3d = np.zeros((arr_2d.shape[0], arr_2d.shape[1], 3), dtype=np.uint8)
    for c, i in palette.items():
        m = arr_2d == c
        arr_3d[m] = i

    return arr_3d
    
    
tst_dataset =  test_dataset('./unet_train/test/', transform=transform)
tst_dataloader = DataLoader(tst_dataset, batch_size=1, num_workers=0, shuffle=False)

model = torch.load('model_16.pth')
model.eval()
for i, image in enumerate(tst_dataloader):
    image = image.to('cuda').float()
    outputs = model(image)
    outputs = outputs.cpu().detach().squeeze().numpy()
    print(outputs)
    #outputs = outputs > 0.5
    
    #RGB = convert_to_color(outputs)
    #io.imsave('./unet_train/pre_{}.png'.format(i), RGB)
    
#outputs = (outputs>0.5) * 255


[[0.11257628 0.03616188 0.02804123 ... 0.02947    0.03467409 0.10364516]
 [0.04674973 0.00798338 0.00509478 ... 0.00591495 0.0074211  0.04167047]
 [0.04395681 0.00666742 0.00372995 ... 0.00461472 0.00634709 0.04029362]
 ...
 [0.0498231  0.00814993 0.00507668 ... 0.00788751 0.00966827 0.05611226]
 [0.05469604 0.01023869 0.00728711 ... 0.0102478  0.01153989 0.05756445]
 [0.12737349 0.0475118  0.04091154 ... 0.05024809 0.05177438 0.13881734]]
[[0.11158665 0.03492364 0.02709022 ... 0.0282689  0.03314438 0.10267923]
 [0.04709937 0.00775517 0.00496569 ... 0.00591159 0.00735112 0.04264004]
 [0.04440732 0.0064431  0.00360508 ... 0.00469352 0.00635594 0.04223432]
 ...
 [0.04103954 0.00585152 0.00354172 ... 0.01046022 0.01188981 0.06026326]
 [0.04732029 0.00795202 0.0055149  ... 0.01298653 0.01361096 0.06062268]
 [0.11742714 0.04095023 0.03464228 ... 0.05705291 0.05665679 0.14229617]]
[[0.11845004 0.03752591 0.02986662 ... 0.03222935 0.0371578  0.10828361]
 [0.0493969  0.00791963 0.00519598 ... 